# San Francisco Housing Rental Analysis

In this assignment, you will perform basic analysis for the San Francisco Housing Market to allow potential real estate investors to choose rental investment properties. 

In [ ]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

In [ ]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.head()

- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function. 

Hint: Use the Pandas groupby function

Optional challenge: Use the min, max, and std to scale the y limits of the chart.

In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby) 
avg_house_per_year = sfo_data['housing_units'].groupby(by='year', level=None).mean()
avg_house_per_year

In [ ]:
# Use the Pandas plot function to plot the average housing units per year.
# Note: You will need to manually adjust the y limit of the chart using the min and max values from above.
avg_house_per_year_plot = avg_house_per_year.plot.bar(ylim=[370000, 387500],
                            title='Housing Units in San Francisco from 2010 to 2016')

avg_house_per_year_plot.set_xlabel("Year")
avg_house_per_year_plot.set_ylabel("Housing Units")

# Optional Challenge: Use the min, max, and std to scale the y limits of the chart
# YOUR CODE HERE!


plt.show()
plt.close(fig_housing_units)

- - - 

## Average Prices per Square Foot

In this section, you will calculate the average gross rent and average sales price for each year. Plot the results as a line chart.

### Average Gross Rent in San Francisco Per Year

In [ ]:
# Calculate the average gross rent and average sale price per square foot
avg_gross_avg_sale_sqft = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby(by='year', level=None).mean()
avg_gross_avg_sale_sqft

In [ ]:
# Plot the Average Gross Rent per Year as a Line Chart 
avg_gross_rent_year = sfo_data['gross_rent'].groupby(by='year', level=None).mean()
avg_gross_rent_year_plot = avg_gross_rent_year.plot.line(ylim=[1000, 4500],
                            title='Average Gross Rent in San Francisco')

avg_gross_rent_year_plot.set_xlabel("Year")
avg_gross_rent_year_plot.set_ylabel("Gross Rent")

### Average Sales Price per Year

In [ ]:
# Plot the Average Sales Price per Year as a line chart
avg_sale_price_year = sfo_data['sale_price_sqr_foot'].groupby(by='year', level=None).mean()
avg_sale_price_year_plot = avg_sale_price_year.plot.line(ylim=[300, 700],
                            title='Average Sales Price per Year in San Francisco')

avg_sale_price_year_plot.set_xlabel("Year")
avg_sale_price_year_plot.set_ylabel("Avg Sale Price")

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create an interactive visulization of the Average Prices with a dropdown selector for the neighborhood.

Hint: It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
# Group by year and neighborhood and then create a new dataframe of the mean values
sfo_data_2 = pd.read_csv(file_path)
avg_prices_df = sfo_data_2.groupby(by=['year','neighborhood'])
avg_prices_df.head()

In [ ]:
# Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
avg_price_mean = avg_prices_df.mean()

avg_prices_df_plot = avg_price_mean.hvplot.line(
    x='year',
    y='sale_price_sqr_foot',
    xlabel= 'Year',
    ylabel= 'Price per Sq Ft',
    title='Average Price per Sq Ft',
    groupby='neighborhood',
    width=600
).opts(yformatter='%0f')

avg_prices_df_plot

- - - 

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods
sfo_data_3 = pd.read_csv(file_path, index_col='neighborhood')
mean_sale_price_neighborhood = sfo_data_3.groupby(by=['neighborhood']).mean()
mean_sale_price_neighborhood.sort_values(by='sale_price_sqr_foot', ascending=False, inplace=True)
mean_sale_price_neighborhood.reset_index(inplace=True)
mean_sale_price_neighborhood = mean_sale_price_neighborhood.head(10)
mean_sale_price_neighborhood.drop(columns='year', inplace=True)
mean_sale_price_neighborhood

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
top_10_neighborhoods_plot = mean_sale_price_neighborhood.hvplot.bar(x='neighborhood',
                                y='sale_price_sqr_foot',
                                rot=45,
                                ylim=[600,1000],
                                title='Top 10 expensive neighborhoods')
top_10_neighborhoods_plot

- - - 

## Parallel Coordinates and Parallel Categories Analysis

In this section, you will use plotly express to create parallel coordinates and parallel categories visualizations so that investors can interactively filter and explore various factors related to the sales price of the neighborhoods. 

Using the DataFrame of Average values per neighborhood (calculated above), create the following visualizations:
1. Create a Parallel Coordinates Plot
2. Create a Parallel Categories Plot

In [ ]:
# Parallel Coordinates Plot
parallel_coordinate_plot = px.parallel_coordinates(
    mean_sale_price_neighborhood,
    color='sale_price_sqr_foot')
parallel_coordinate_plot

In [ ]:
# Parallel Categories Plot
parallel_categories_plot = px.parallel_categories(
mean_sale_price_neighborhood,
color='sale_price_sqr_foot',
dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
color_continuous_scale=px.colors.sequential.Inferno)
parallel_categories_plot

- - - 

## Neighborhood Map

In this section, you will read in neighboor location data and build an interactive map with the average prices per neighborhood. Use a scatter_mapbox from plotly express to create the visualization. Remember, you will need your mapbox api key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)
df_neighborhood_locations.head()

### Data Preparation

You will need to join the location data with the mean prices per neighborhood

1. Calculate the mean values for each neighborhood
2. Join the average values with the neighborhood locations

In [ ]:
# Calculate the mean values for each neighborhood
all_neighborhoods = sfo_data.groupby(
    by='neighborhood').mean()
all_neighborhoods.reset_index(inplace=True)
all_neighborhoods.rename(columns={'neighborhood': 'Neighborhood'},
                         inplace=True)
all_neighborhoods.head()

In [ ]:
# Join the average values with the neighborhood locations
all_neighborhoods = pd.merge(
    df_neighborhood_locations,
    all_neighborhoods,
    on='Neighborhood')
all_neighborhoods.head()

### Mapbox Visualization

Plot the aveage values per neighborhood with a plotly express scatter_mapbox visualization.

In [ ]:
# Create a scatter mapbox to analyze neighborhood info
px.set_mapbox_access_token(mapbox_token)
map_scatter = px.scatter_mapbox(all_neighborhoods,
    lat='Lat',
    lon='Lon',
    size='sale_price_sqr_foot',
    color='gross_rent',
    color_continuous_scale=px.colors.cyclical.IceFire,
    size_max=15,
    zoom=11,
    hover_name='Neighborhood',
    title='Averange Sale Price Per Square Foot and Gross Rent in San Francisco',
)
map_scatter.show()